# Creating a model:

In [ ]:
%pylab inline
%matplotlib notebook

import importlib
import odecell
importlib.reload(odecell)

model = odecell.modelbuilder.MetabolicModel()
print( "Base rate for zero order reaction: \n", model.zeroOrder.getBaseRate() )
print()
print( "Base rate for first order reaction: \n", model.firstOrder.getBaseRate() )
print()
print( "Base rate for second order reaction: \n", model.secondOrder.getBaseRate() )
print()
print( "Base rate for first order Michaelis-Menten reaction: \n", model.firstOrderMM.getBaseRate() )
print()
print( "Base rate for second order Michaelis-Menten reaction: \n", model.secondOrderMM.getBaseRate() )
print()
print( "Base rate for reversible random Bi Bi reaction: \n", model.randomBiBiRev.getBaseRate() )

In [ ]:
print( "Final rate form: \n", model.firstOrderMM.getRate({"Km": 2.134, "Vmax":5.67,"Sub1": "Glu"}) )
print()
print( "Final rate form: \n", model.firstOrderMM.getRate({"Km": 0.23, "Vmax":2.34,"Sub1": "G6P"}) )

In [ ]:
model.printAvailableForms()

In [ ]:
model.testOrder = odecell.modelbuilder.RateForm("$Sub1*$Par1/($Sub2/$Par2)")
model.updateAvailableForms()

In [ ]:
model.printAvailableForms() #This function also updates the internal dictionary of rate forms.

In [ ]:
print( "Final rate form: \n", model.testOrder.getRate({"Par1": 0.23, "Par2":2.34,"Sub1": "G6P","Sub2":"PEP"}) )

# Adding  metabolites and reactions

In [ ]:
metIndx = model.addMetabolite("Glc_out","Extracellular Glucose", 0.0647)
metIndx = model.addMetabolite("Glc_in","Intracellular Glucose", 0.1)
metIndx = model.addMetabolite("PEP","Phosphoenolpyruvate", 0.01)
metIndx = model.addMetabolite("G6P","Glucose-6-phosphate", 0.02)
#model.parseMetabolites("metabolites.csv")

In [ ]:
rxnIndx = model.addReaction("PTSout","secondOrderMM","PTS for extracellulat glucose")

In [ ]:
print( model.getReaction(rxnIndx) )

In [ ]:
#model.addSubstrate(rxnIndx, "Sub1", "Glc_out")
model.addSubstrate(rxnIndx, "Sub2", "PEP")
model.addParameter(rxnIndx, "Sub1", 0.0647)
model.addProduct("PTSout","Prod1","G6P")

In [ ]:
paramDict = {"Vmax": 22, "Ki1":0.005, "Km1":0.005, "Km2":0.3}
for key,val in paramDict.items():
    model.addParameter("PTSout", key, val)

#model.parseParameters("parameters.csv")

### -> Checking your work

In [ ]:
print( model.getReaction(rxnIndx) )

In [ ]:
model.prepModel()
model.getReaction(rxnIndx).getFinalRate()

### -> Expanding the example

In [ ]:
model.addMetabolite("ATP","Adenosine triphosphate", 0.1)
model.addMetabolite("ADP","Adenosine diphosphate", 0.1)

rxnIndx = model.addReaction("PTSin","secondOrderMM","PTS for instracellular glucose")
model.addSubstrate(rxnIndx, "Sub1", "Glc_in")
model.addSubstrate(rxnIndx, "Sub2", "PEP")
model.addProduct(rxnIndx,"Prod1","G6P")
paramDict = {"Vmax": 22, \
                "Ki1":0.005, \
                "Km1":0.005, \
                "Km2":0.3}
for key,val in paramDict.items():
    model.addParameter(rxnIndx, key, val)

rxnIndx = model.addReaction("GluMgl","firstOrderMM","Glucose transport through Mgl")
model.addParameter(rxnIndx, "Sub1", 0.0647)
model.addProduct(rxnIndx,"Prod1","Glc_in")
paramDict = {"Vmax": 10, \
             "Km":0.0002}
for key,val in paramDict.items():
    model.addParameter(rxnIndx, key, val)

rxnIndx = model.addReaction("Glk","randomBiBiRev","Glucokinase")
model.addSubstrate(rxnIndx, "Sub1", "Glc_in")
model.addSubstrate(rxnIndx, "Sub2", "ATP")
model.addProduct(rxnIndx,"Prod1","G6P")
model.addProduct(rxnIndx,"Prod2","ADP")
    
paramDict = {"Vmax": 10, \
                "Keq":20, \
                "Km1":3.76, \
                "Km2":0.78, \
                "Kmp1":100, \
                "Kmp2":100, \
                "Kd1":100, \
                "pHdep":1 }
for key,val in paramDict.items():
    model.addParameter(rxnIndx, key, val)


# Creating a solver object

In [ ]:
from scipy import integrate
import importlib
from odecell import modelbuilder, solver, paropt
importlib.reload(modelbuilder)
importlib.reload(solver)
importlib.reload(paropt)

In [ ]:
solv = solver.ModelSolver(model)

In [ ]:
solv.buildCall(odeint=False, verbose=2)

In [ ]:
integrator = integrate.ode(solv)
integrator.set_initial_value(model.getInitVals())

In [ ]:
totalTime = 0.1
step = 0.01
print(model.getInitVals())
while integrator.successful() and integrator.t < totalTime:
    print(integrator.t, integrator.integrate(integrator.t + step))

# A toy model: 
## A + B -> C

In [ ]:
from scipy import integrate
import numpy as np
import importlib
from odecell import modelbuilder, solver, paropt
importlib.reload(modelbuilder)
importlib.reload(solver)
importlib.reload(paropt)

print("Building the model...\n")
model = modelbuilder.MetabolicModel()
model.setVerbosity(2)

# In particles
model.addMetabolite("A","A name", 1000)
model.addMetabolite("B","B name", 800)
model.addMetabolite("C","C name", 0)

rxnIndx = model.addReaction("Forward","secondOrder","Forward reaction")
model.addSubstrate(rxnIndx, "Sub1", "A")
model.addSubstrate(rxnIndx, "Sub2", "B")
model.addProduct(rxnIndx,"Prod1","C")
# Rate of 10^6 M/s or 0.001 molecules/second in a 1 fL volume
paramDict = {"K": 0.001}
for key,val in paramDict.items():
    model.addParameter(rxnIndx, key, val)

rxnIndx = model.addReaction("Reverse","firstOrder","Reverse reaction")
model.addSubstrate(rxnIndx, "Sub1", "C")
model.addProduct(rxnIndx, "Prod1", "A")
model.addProduct(rxnIndx,"Prod2","B")
# Rate of 10^8 M/s or 0.001 molecules/second in a 1 fL volume
paramDict = {"K": 0.1}
for key,val in paramDict.items():
    model.addParameter(rxnIndx, key, val)

## Create the solver object

In [ ]:
print("\nBuilding the solver...\n")
    
solv = solver.ModelSolver(model)
solv.buildCall(odeint=False, useJac=True, verbose=2)

## Integrate the ODE model over time

We use SciPy's `ode` module to integrate our ODE model.

In [ ]:
integrator = integrate.ode(solv, solv.calcJac)
integrator.set_initial_value(model.getInitVals())
totalTime = 10
step = 0.1

print("Initial concentration: ", model.getInitVals())

results = np.empty((0,len(model.getInitVals())), float)

while integrator.successful() and integrator.t < totalTime:
    currConcentration = integrator.integrate(integrator.t + step)
    #print(integrator.t, currConcentration)
    results = np.append(results, [np.asarray(currConcentration)], axis=0 )
    
print("Final concentration at time " + str(totalTime) + ":", currConcentration)


# Plotting your results

We now add a simple `matplotlib` visualization of the time evolution of the system.

In [ ]:
%matplotlib notebook

import matplotlib
import matplotlib.pyplot as plt

In [ ]:
t = np.arange(0,len(results),1)
plt.plot(t, results[:,0], 'r-',\
         t, results[:,1], 'g-',\
         t, results[:,2], 'b--')

## Dynamically alter conditions of the model, and the model itself

We can alternate model integration with modifications of model conditions and parameters.

In [ ]:
integrator = integrate.ode(solv)
integrator.set_initial_value(model.getInitVals())
totalTime = 10
step = 0.1

print("Initial concentration: ", model.getInitVals())

results = np.empty((0,len(model.getInitVals())), float)

##################
## Run initial model.
##################

while integrator.successful() and integrator.t < totalTime:
    currConcentration = integrator.integrate(integrator.t + step)
    #print(integrator.t, currConcentration)
    results = np.append(results, [np.asarray(currConcentration)], axis=0 )

##################
## Apply change in the model environment!
##################

# Re-initialize solver object and integrator
solv.buildCall(odeint=False, verbose=1, cythonBuild=False)
integrator = integrate.ode(solv)

# Get the concentrations of all metabolites from the end of the initial
#   execution, then add 300 "units" (copies, mM, M, etc) of metabolite "C".
integrator.set_initial_value(results[-1,:] + [0,0,300])

# Run for another 10 units of time. (Units of time depend on choice of units for model parameters)
totalTime = 10
while integrator.successful() and integrator.t < totalTime:
    currConcentration = integrator.integrate(integrator.t + step)
    #print(integrator.t, currConcentration)
    results = np.append(results, [np.asarray(currConcentration)], axis=0 )

##################
## Apply change in the model itself!
##################

# Modify model parameter from 0.1 to 0.5
model.addParameter("Reverse", "K", 0.5)

# Re-initialize solver object and integrator
solv = solver.ModelSolver(model)
solv.buildCall(odeint=False, verbose=1, cythonBuild=False)
integrator = integrate.ode(solv)

# Start the simulation from the final values of the previous run.
integrator.set_initial_value(results[-1,:])

# Run for another 10 units of time.
totalTime = 10
while integrator.successful() and integrator.t < totalTime:
    currConcentration = integrator.integrate(integrator.t + step)
    #print(integrator.t, currConcentration)
    results = np.append(results, [np.asarray(currConcentration)], axis=0 )
    
print("Final concentration at time " + str(totalTime) + ":", currConcentration)

In [ ]:
t = np.arange(0,len(results),1)
plt.plot(t, results[:,0], 'r-',\
         t, results[:,1], 'g-',\
         t, results[:,2], 'b--')

## Use CVODE to integrate complex models

We can also use the `pycvodes` interface for the C++ Sundials integrator `cvode`.

In [ ]:
from pycvodes import integrate_predefined  # also: integrate_adaptive

In [ ]:
# Re-initialize solver object and integrator
model.addParameter("Reverse", "K", 0.1)
solv = solver.ModelSolver(model)
solv.buildCall(odeint=False, verbose=1, useJac=True, cythonBuild=False)

# Define a wrapper for the function call. CVODE wants the deltas to be written in an array passed as an argument.
def f_wrap(t, y, dydt):
    result = solv(t, np.asarray(y))
    dydt[:] = result[:]
    
def j_wrap(t, y, Jmat, dfdt=None, fy=None):
    result = solv.calcJac(t, np.asarray(y))
    Jmat[:,:] = result[:,:]

In [ ]:
y0 = model.getInitVals()
dt0=1e-8; t0=0.0; atol=1e-8; rtol=1e-8
tout = np.linspace(0, totalTime, 200)

yout, info = integrate_predefined(f_wrap, j_wrap, y0, tout, atol, rtol, dt0, method='bdf')

In [ ]:
t = np.arange(0,len(yout),1)
plt.plot(t, yout[:,0], 'r-',\
         t, yout[:,1], 'g-',\
         t, yout[:,2], 'b--')

## Use Cython to speed up integration

ODECell can compile the right hand side (RHS) of the ODE system with `cython` so the code is compiled to C++, speeding up the integration time of complex models.

In [ ]:
# Re-initialize solver object and integrator
model.addParameter("Reverse", "K", 1)
solv = solver.ModelSolver(model)
solv.buildCall(odeint=False, verbose=1, useJac=True, cythonBuild=True)

# Define a wrapper for the function call. CVODE wants the deltas to be written in an array passed as an argument.
def f_wrap(t, y, dydt):
    result = solv(t, np.asarray(y))
    dydt[:] = result[:]
    
def j_wrap(t, y, Jmat, dfdt=None, fy=None):
    result = solv.calcJac(t, np.asarray(y))
    Jmat[:,:] = result[:,:]

In [ ]:
y0 = model.getInitVals()
dt0=1e-8; t0=0.0; atol=1e-8; rtol=1e-8
tout = np.linspace(0, totalTime, 200)

yout, info = integrate_predefined(f_wrap, j_wrap, y0, tout, atol, rtol, dt0, method='bdf')

In [ ]:
t = np.arange(0,len(yout),1)
plt.plot(t, yout[:,0], 'r-',\
         t, yout[:,1], 'g-',\
         t, yout[:,2], 'b--')